In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

In [ ]:
# load the roster (for race + gender info)
roster = pd.read_csv('../final/roster.csv')
roster.head()

In [ ]:
# load the awards data
awards  = pd.read_csv('../final/awards.csv')
awards.head()

In [ ]:
# create a map of shortnames of races
races_short = {'WHITE' : 'White', 
              'BLACK' : 'Black',
              'BLACK HISPANIC' : 'Bl. Hisp.',
              'WHITE HISPANIC' : 'Wh. Hisp.',
              'ASIAN/PACIFIC ISLANDER' : 'Asian/P.I.',
              'AMER IND/ALASKAN NATIVE' : 'Indig.',
              'Other' : 'Other'}

In [ ]:
# build a map of UID -> (race, gender)
trait_map = {}
traits = ['race', 'gender']

for i in range(roster.shape[0]):
    trait_map[roster.uid.iloc[i]] = {tr : roster[tr].iloc[i] for tr in traits}

# for each trait, add a column to the awards data using the map
for tr in traits:
    awards[tr] = awards.uid.map(lambda x : trait_map[x][tr])
    
# in the below plots, Asian/Pacific Islander, Indigenous, and Black Hispanic categories are too small to get reliable awards ratios.
# group them into "Other"
awards.race = awards.race.map(lambda x : 'Other' if x in ['ASIAN/PACIFIC ISLANDER', 'AMER IND/ALASKAN NATIVE', 'BLACK HISPANIC'] else x)
roster.race = roster.race.map(lambda x : 'Other' if x in ['ASIAN/PACIFIC ISLANDER', 'AMER IND/ALASKAN NATIVE', 'BLACK HISPANIC'] else x)

In [ ]:
# aggregate the number of unique officers in each (race, gender) category
num_offs = roster.groupby(['race', 'gender']).agg(['count'])
num_offs.reset_index(inplace=True)
num_offs['count'] = num_offs['uid']['count']
# fix tuple colnames
num_offs.columns = num_offs.columns.map(''.join)
# restrict the dataframe to just (race, gender, count)
num_offs = num_offs[['race', 'gender', 'count']]
num_offs = num_offs.rename(columns = {'count' : 'officers'})
# remove the 'X' gender (by visual inspection, these should be missing data -- not nonbinary genders)
num_offs = num_offs[num_offs.gender != 'X']
# replace races with shortnames
num_offs.race = num_offs.race.map(races_short)
num_offs.reset_index(inplace=True)
num_offs

In [ ]:
# aggregate the number of award requests in each (race, gender) category
num_awds = awards.groupby(['race', 'gender']).agg(['count'])
num_awds.reset_index(inplace=True)
num_awds['count'] = num_awds['uid']['count']
# fix tuple colnames
num_awds.columns = num_awds.columns.map(''.join)
# restrict to just these columns
num_awds = num_awds[['race', 'gender', 'count']]
num_awds = num_awds.rename(columns = {'count' : 'awards'})
# replace races with shortnames
num_awds.race = num_awds.race.map(races_short)
num_awds.reset_index(inplace=True)
num_awds

In [ ]:
# join the two data frames and remove duplicated columns
cc = pd.concat([num_offs, num_awds], axis=1)
cc = cc.loc[:, ~cc.columns.duplicated()]
# compute the number of awards per officer in each category
cc['frac'] = cc['awards']/cc['officers']
cc

In [ ]:
# Load the salary data
salary  = pd.read_csv('../final/salary.csv')
salary.head()

# create a mapping of position shortnames
positions_short = {'POLICE OFFICER' : 'Officer', 
             'POLICE OFFICER (ASSIGNED AS DETECTIVE)' : 'Detective',
             'SERGEANT' : 'Sergeant', 
             'LIEUTENANT' : 'Lieutenant', 
             'CAPTAIN' : 'Captain', 
             'COMMANDER' : 'Commander', 
             'DEPUTY CHIEF' : 'Dep. Chief', 
             'CHIEF' : 'Chief',
             'DEPUTY SUPERINTENDENT' : 'Dep. Sup.', 
             'FIRST DEPUTY SUPERINTENDENT' : '1st Dep. Sup.'}

# create a map of shortnames of races
races_short = {'WHITE' : 'White', 
              'BLACK' : 'Black',
              'BLACK HISPANIC' : 'Bl. Hisp.',
              'WHITE HISPANIC' : 'Wh. Hisp.',
              'ASIAN/PACIFIC ISLANDER' : 'Asian/P.I.',
              'AMER IND/ALASKAN NATIVE' : 'Indig.',
              'Other' : 'Other'}

# build a map of UID -> (race, gender)
trait_map = {}
traits = ['race', 'gender']

for i in range(roster.shape[0]):
    trait_map[roster.uid.iloc[i]] = {tr : roster[tr].iloc[i] for tr in traits}

# for each trait, add a column to the awards data using the map
for tr in traits:
    salary[tr] = salary.uid.map(lambda x : trait_map[x][tr])
    
# in the below plots, Asian/Pacific Islander, Indigenous, and Black Hispanic categories are too small to visualize individually.
# group them into "Other"
salary.race = salary.race.map(lambda x : 'Other' if x in ['ASIAN/PACIFIC ISLANDER', 'AMER IND/ALASKAN NATIVE', 'BLACK HISPANIC'] else x)
       
salary

In [ ]:
# generate the plot
# sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')

plt.figure(figsize = (20,5.5))


plt.subplot(121)
# plot salary vs years in posn for these positions
positions = ['POLICE OFFICER', 'POLICE OFFICER (ASSIGNED AS DETECTIVE)', 'SERGEANT', 'LIEUTENANT', 'COMMANDER']

for posn in positions:
    # restrict to the particular position
    rows = salary[salary.position_description == posn]
    
    # compute the number of years the officer has spent in their current position for each record
    rows['posn_start_year'] = list(map(lambda x : x.year, pd.to_datetime(rows.present_posn_start_date.values)))
    rows['years_in_posn'] = rows['year'] - rows['posn_start_year']   

    # remove very old entries
    rows = rows[rows.years_in_posn <= 30]
    # aggregate and compute stats
    agg = rows.groupby('years_in_posn').salary.agg(['mean', 'std', 'count'])
    agg.reset_index(inplace=True)
    # only plot when there were a large enough group of officers (at least 3)
    agg = agg[agg['count'] >= 3]
    plt.errorbar(agg.index, agg['mean'], agg['std'], label=positions_short[posn], lw = 3)
plt.xlabel('Years in Position', fontsize = 30)
plt.ylabel('Salary (USD)', fontsize = 30)
plt.xticks(fontsize = 25)
plt.yticks(fontsize = 25)

plt.legend(fontsize = 18)

plt.subplot(122)


ax = sns.barplot(x = 'race', y = 'frac', hue = 'gender', data = cc)
plt.xticks(fontsize =30)
# plt.xticks(rotation=45, ha="right", rotation_mode="anchor")
plt.ylabel('Award Requests per Officer', fontsize = 25)
plt.xlabel('CPD Race Category', fontsize = 30)
plt.yticks(fontsize = 25)
plt.legend(fontsize = 25)
ax.get_legend().set_title('CPD Gender', prop={'size':25})
plt.tight_layout()
plt.savefig('../doc/figs/salary_awards.pdf', bbox_inches = 'tight', dpi = 1000)